In [1]:
import base64

In [2]:
TYPE_TEMP         = 0x01 #temp 2 bytes -3276.8°C -->3276.7°C
TYPE_RH           = 0x02 #Humidity 1 byte  0-100%
TYPE_ACC          = 0x03 #acceleration 3 bytes X,Y,Z -128 --> 127 +/-63=1G
TYPE_LIGHT        = 0x04 #Light 2 bytes 0-->65535 Lux
TYPE_MOTION       = 0x05 #No of motion 1 byte  0-255
TYPE_CO2          = 0x06 #Co2 2 bytes 0-65535 ppm 
TYPE_VDD          = 0x07 #VDD 2byte 0-65535mV
TYPE_ANALOG1      = 0x08 #VDD 2byte 0-65535mV
TYPE_GPS          = 0x09 #3bytes lat 3bytes long binary
TYPE_PULSE1       = 0x0A #2bytes relative pulse count
TYPE_PULSE1_ABS   = 0x0B #4bytes no 0->0xFFFFFFFF
TYPE_EXT_TEMP1    = 0x0C #2bytes -3276.5C-->3276.5C
TYPE_EXT_DIGITAL  = 0x0D #1bytes value 1 or 0
TYPE_EXT_DISTANCE = 0x0E     #2bytes distance in mm
TYPE_ACC_MOTION   = 0x0F     #1byte number of vibration/motion
TYPE_IR_TEMP      = 0x10     #2bytes internal temp 2bytes external temp -3276.5C-->3276.5C
TYPE_OCCUPANCY    = 0x11     #1byte data
TYPE_WATERLEAK    = 0x12     #1byte data 0-255 
TYPE_GRIDEYE      = 0x13     #65byte temperature data 1byte ref+64byte external temp
TYPE_PRESSURE     = 0x14     #4byte pressure data (hPa)
TYPE_SOUND        = 0x15     #2byte sound data (peak/avg)
TYPE_PULSE2       = 0x16     #2bytes 0-->0xFFFF
TYPE_PULSE2_ABS   = 0x17     #4bytes no 0->0xFFFFFFFF
TYPE_ANALOG2      = 0x18     #2bytes voltage in mV
TYPE_EXT_TEMP2    = 0x19     #2bytes -3276.5C-->3276.5C
TYPE_EXT_DIGITAL2 = 0x1A     # 1bytes value 1 or 0 
TYPE_EXT_ANALOG_UV= 0x1B     # 4 bytes signed int (uV)
TYPE_DEBUG        = 0x3D     # 4bytes debug 

In [3]:
def bin8dec(bin):
    num=bin&0xFF;
    if (0x80 & num): 
        num = - (0x0100 - num);
    return num

In [157]:
def bin16dec(bin):
    num=bin&0xFFFF;
    if (0x8000 & num):
        num = - (0x010000 - num);
    print("bin16dec", bin,"-->",num);
    return num

In [5]:
def hexToBytes(hex):
    bytes = []
    for c in range(0,len(hex),2):
        bytes.append(int(hex[c: c+2],16))
    return bytes

In [6]:
def b64ToHex(b64):
    return base64.b64decode(b64).hex()

In [242]:
def DecodeElsysPayload(data):
    obj = {};
    obj["device"]="elsys"
    print("data ",data," len ",len(data))
    i=0
    while(i<len(data)):
        #Temperature
        if data[i] == TYPE_TEMP:    
            temp=(data[i+1]<<8)|(data[i+2])
            print("temp")
            temp=bin16dec(temp)
            obj["temperature"]=temp/10
            print(i, "before")
            i=i+2
            print(i, "after")
        
        #Humidity
        elif data[i] == TYPE_RH:    
            rh=(data[i+1])
            obj["humidity"]=rh
            i+=1
            
        #Acceleration
        elif data[i] == TYPE_ACC:    
            obj["x"]=bin8dec(data[i+1])
            obj["y"]=bin8dec(data[i+2])
            obj["z"]=bin8dec(data[i+3])
            i+=3
            
        #Light 
        elif data[i] == TYPE_LIGHT: 
            obj["light"]=(data[i+1]<<8)|(data[i+2])
            i+=2
            
        #Motion sensor(PIR)
        elif data[i] == TYPE_MOTION:
            obj["motion"]=(data[i+1])
            i+=1
            
        #CO2
        elif data[i] == TYPE_CO2:
            obj["co2"]=(data[i+1]<<8)|(data[i+2])
            i+=2
            
        #Battery level
        elif data[i] == TYPE_VDD:
            obj["vdd"]=(data[i+1]<<8)|(data[i+2])
            i+=2
            
        #Analog input 1
        elif data[i] == TYPE_ANALOG1: 
            obj["analog1"]=(data[i+1]<<8)|(data[i+2])
            i+=2
            
        #gps
        elif data[i] ==  TYPE_GPS:
            obj["lat"]=(data[i+1]<<16)|(data[i+2]<<8)|(data[i+3])
            obj["lng"]=(data[i+4]<<16)|(data[i+5]<<8)|(data[i+6])
            i+=6
            
        #Pulse input 1
        elif data[i] == TYPE_PULSE1:    
            obj["pulse1"]=(data[i+1]<<8)|(data[i+2])
            i+=2
            
        #Pulse input 1 absolute value
        elif data[i] ==  TYPE_PULSE1_ABS: 
            pulseAbs=(data[i+1]<<24)|(data[i+2]<<16)|(data[i+3]<<8)|(data[i+4])
            obj["pulseAbs"]=pulseAbs
            i+=4
            
        #External temp
        elif data[i] ==TYPE_EXT_TEMP1:
            temp=(data[i+1]<<8)|(data[i+2])
            print("ex_temp")
            temp=bin16dec(temp)
            obj["externalTemperature"]=temp/10
            i+=2
            
        #Digital input
        elif data[i] == TYPE_EXT_DIGITAL:
            obj["digital"]=(data[i+1])
            i+=1
            
        #Distance sensor input 
        elif data[i] == TYPE_EXT_DISTANCE:     
            obj["distance"]=(data[i+1]<<8)|(data[i+2])
            obj["JBJB"]="my debug here"
            i+=2
            
        #Acc motion
        elif data[i] == TYPE_ACC_MOTION:     
            obj["accMotion"]=(data[i+1])
            i+=1
            
        #IR temperature
        elif data[i] == TYPE_IR_TEMP:
            iTemp=(data[i+1]<<8)|(data[i+2])
            print("ir_temp")
            iTemp=bin16dec(iTemp)
            eTemp=(data[i+3]<<8)|(data[i+4])
            eTemp=bin16dec(eTemp)
            obj["irInternalTemperature"]=iTemp/10
            obj["irExternalTemperature"]=eTemp/10
            i+=4
            
        #Body occupancy
        elif data[i] == TYPE_OCCUPANCY:
            obj["occupancy"]=(data[i+1])
            i+=1
            
        #Water leak
        elif data[i] == TYPE_WATERLEAK:
            obj["waterleak"]=(data[i+1])
            i+=1
            
        #Grideye data
        elif data[i] == TYPE_GRIDEYE:
            obj["grideye"]="8x8 missing"
            i+=65
            
        #External Pressure
        elif data[i] == TYPE_PRESSURE:
            temp=(data[i+1]<<24)|(data[i+2]<<16)|(data[i+3]<<8)|(data[i+4])
            obj["pressure"]=temp/1000
            i+=4
            
        #Sound
        elif data[i] == TYPE_SOUND:
            obj["soundPeak"]=data[i+1]
            obj["soundAvg"]=data[i+2]
            i+=2
            
        #Pulse 2
        elif data[i] == TYPE_PULSE2:
            obj["pulse2"]=(data[i+1]<<8)|(data[i+2])
            i+=2
            
        #Pulse input 2 absolute value
        elif data[i] == TYPE_PULSE2_ABS:
            obj["pulseAbs2"]=(data[i+1]<<24)|(data[i+2]<<16)|(data[i+3]<<8)|(data[i+4])
            i+=4
            
        #Analog input 2
        elif data[i] ==  TYPE_ANALOG2:   
            obj["analog2"]=(data[i+1]<<8)|(data[i+2])
            i+=2
            
        #External temp 2
        elif data[i] == TYPE_EXT_TEMP2:
            temp=(data[i+1]<<8)|(data[i+2])
            print(ex_temp2)
            temp=bin16dec(temp)
            obj["externalTemperature2"]=temp/10
            i+=2
            
        #Digital input 2 
        elif data[i] ==  TYPE_EXT_DIGITAL2:
            obj["digital2"]=(data[i+1])
            i+=1
             
        #Load cell analog uV
        elif data[i] == TYPE_EXT_ANALOG_UV:
            obj["analogUv"] = (data[i + 1] << 24) | (data[i + 2] << 16) | (data[i + 3] << 8) | (data[i + 4])
            i += 4
            
        else:
            #print("something is wrong with data")
            i=len(data)
        i+=1
    return obj;


In [243]:
hex_string=b64ToHex("AQDsAiMDPBH7Bw4uCwAAAJkNAA8AEgA= ")
hex_string

'0100ec0223033c11fb070e2e0b000000990d000f001200'

In [244]:
data=hexToBytes(hex_string)
data

[1,
 0,
 236,
 2,
 35,
 3,
 60,
 17,
 251,
 7,
 14,
 46,
 11,
 0,
 0,
 0,
 153,
 13,
 0,
 15,
 0,
 18,
 0]

In [245]:
message_bytes = base64.b64decode("AQDsAiMDPBH7Bw4uCwAAAJkNAA8AEgA= ")
message_bytes

b'\x01\x00\xec\x02#\x03<\x11\xfb\x07\x0e.\x0b\x00\x00\x00\x99\r\x00\x0f\x00\x12\x00'

In [246]:
DecodeElsysPayload(data)

data  [1, 0, 236, 2, 35, 3, 60, 17, 251, 7, 14, 46, 11, 0, 0, 0, 153, 13, 0, 15, 0, 18, 0]  len  23
temp
bin16dec 236 --> 236
0 before
2 after


{'device': 'elsys',
 'temperature': 23.6,
 'humidity': 35,
 'x': 60,
 'y': 17,
 'z': -5,
 'vdd': 3630,
 'pulseAbs': 153,
 'digital': 0,
 'accMotion': 0,
 'waterleak': 0}

In [247]:
for i in range(0,10):
    if(i==5):
        i+=5
    print(i)

0
1
2
3
4
10
6
7
8
9


In [248]:
DecodeElsysPayload(base64.b64decode("AQC0AigEAAAFAAYBswcORA=="))

data  b'\x01\x00\xb4\x02(\x04\x00\x00\x05\x00\x06\x01\xb3\x07\x0eD'  len  16
temp
bin16dec 180 --> 180
0 before
2 after


{'device': 'elsys',
 'temperature': 18.0,
 'humidity': 40,
 'light': 0,
 'motion': 0,
 'co2': 435,
 'vdd': 3652}

{
  "app_id": "cambridge-sensor-network",
  "dev_id": "elsys-eye-044501",
  "hardware_serial": "A81758FFFE044501",
  "port": 5,
  "counter": 146,
  "payload_raw": 
  ### "BQERAQ=="
  ,
  "payload_fields": {
    "device": "elsys",
    "motion": 1,
    "occupancy": 1
  }

{
  "app_id": "cambridge-sensor-network",
  "dev_id": "elsys-eye-044501",
  "hardware_serial": "A81758FFFE044501",
  "port": 5,
  "counter": 145,
  "payload_raw": 
  ### "AQDdAh8EAN0FAAcOIhEA"
  ,
  "payload_fields": {
    "device": "elsys",
    "humidity": 31,
    "light": 221,
    "motion": 0,
    "occupancy": 0,
    "temperature": 22.1,
    "vdd": 3618
  },


In [13]:
import json

In [33]:
msg='{ "app_id": "cambridge-sensor-network", "dev_id": "elsys-eye-044501", "hardware_serial": "A81758FFFE044501", "port": 5, "counter": 146, "payload_raw":"BQERAQ==", "payload_fields": { "device": "elsys", "motion": 1, "occupancy": 1 }}'

In [34]:
msg

'{ "app_id": "cambridge-sensor-network", "dev_id": "elsys-eye-044501", "hardware_serial": "A81758FFFE044501", "port": 5, "counter": 146, "payload_raw":"BQERAQ==", "payload_fields": { "device": "elsys", "motion": 1, "occupancy": 1 }}'

In [51]:
eye_msg="{'app_id': 'cambridge-sensor-network', 'dev_id': 'ijl20-sodaq-ttn', 'hardware_serial': '763A53FFB342FBD8', 'port': 1, 'counter': 90168, 'payload_raw': 'MjYuNDU=', 'payload_fields': {'device': 'temperature', 'temperature': '26.45'}, 'metadata': {'time': '2020-02-13T15:58:37.526235678Z', 'frequency': 868.5, 'modulation': 'LORA', 'data_rate': 'SF7BW125', 'airtime': 51456000, 'coding_rate': '4/5', 'gateways': [{'gtw_id': 'csn-mtcdtip-003c70', 'gtw_trusted': True, 'timestamp': 3371042907, 'time': '2020-02-13T15:58:37Z', 'channel': 2, 'rssi': -87, 'snr': 7, 'rf_chain': 1, 'latitude': 52.21095, 'longitude': 0.09169465, 'altitude': 15, 'location_source': 'registry'}, {'gtw_id': 'csn-mtcap-014abf', 'gtw_trusted': True, 'timestamp': 3301353435, 'time': '2020-02-13T16:24:43Z', 'channel': 2, 'rssi': -47, 'snr': 6.5, 'rf_chain': 1, 'latitude': 52.21096, 'longitude': 0.09249892, 'location_source': 'registry'}, {'gtw_id': 'csn-mtcdt-00060f', 'gtw_trusted': True, 'timestamp': 119746195, 'time': '2020-02-13T16:04:18Z', 'channel': 2, 'rssi': -36, 'snr': 6.5, 'rf_chain': 1}]}}"

In [52]:
eye_msg

"{'app_id': 'cambridge-sensor-network', 'dev_id': 'ijl20-sodaq-ttn', 'hardware_serial': '763A53FFB342FBD8', 'port': 1, 'counter': 90168, 'payload_raw': 'MjYuNDU=', 'payload_fields': {'device': 'temperature', 'temperature': '26.45'}, 'metadata': {'time': '2020-02-13T15:58:37.526235678Z', 'frequency': 868.5, 'modulation': 'LORA', 'data_rate': 'SF7BW125', 'airtime': 51456000, 'coding_rate': '4/5', 'gateways': [{'gtw_id': 'csn-mtcdtip-003c70', 'gtw_trusted': True, 'timestamp': 3371042907, 'time': '2020-02-13T15:58:37Z', 'channel': 2, 'rssi': -87, 'snr': 7, 'rf_chain': 1, 'latitude': 52.21095, 'longitude': 0.09169465, 'altitude': 15, 'location_source': 'registry'}, {'gtw_id': 'csn-mtcap-014abf', 'gtw_trusted': True, 'timestamp': 3301353435, 'time': '2020-02-13T16:24:43Z', 'channel': 2, 'rssi': -47, 'snr': 6.5, 'rf_chain': 1, 'latitude': 52.21096, 'longitude': 0.09249892, 'location_source': 'registry'}, {'gtw_id': 'csn-mtcdt-00060f', 'gtw_trusted': True, 'timestamp': 119746195, 'time': '2020

In [53]:
msg_json=json.loads(eye_msg)
msg_json

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [54]:
msg_json

{'app_id': 'cambridge-sensor-network',
 'dev_id': 'elsys-eye-044501',
 'hardware_serial': 'A81758FFFE044501',
 'port': 5,
 'counter': 146,
 'payload_raw': 'BQERAQ==',
 'payload_fields': {'device': 'elsys', 'motion': 1, 'occupancy': 1}}

'elsys-eye-044501'

In [24]:
print(msg_json.dev_id)

AttributeError: 'str' object has no attribute 'dev_id'

In [ ]:
def print_msg(client, userdata, message):
    #print("%s : %s" % (message.topic, message.payload))
    inc_msg=str(message.payload,'utf-8')
    msg_json=json.dumps(inc_msg)
    
    print(msg_json["dev_id"])

In [12]:
def decodeAdeunis(bytes):
    decoded={}
    decoded["device"]= "adeunis_test"
    
    offset = 1# index into payload for next field
    
    #// Temperature
    if (bytes[0] & 0x80):  #// temperature present
        decoded['temperature'] = bytes[offset+1]# // note only works for positive temps
    
    
    #// Button press
    if (bytes[0] & 0x20): # // Button was pressed
        decoded['button'] = true;
    
    #// GPS
    if (bytes[0] & 0x10): # // gps present
        degrees = (bytes[offset] >> 4) * 10 + (bytes[offset] & 0x0F) ;
      #//decoded['lat_deg'] = degrees;
      
        minutes = (bytes[offset+1] >> 4) * 10 + (bytes[offset+1] & 0x0F) +(bytes[offset+2] >> 4) / 10 + (bytes[offset+2] & 0x0F) / 100 + (bytes[offset+3] >> 4) / 1000;
          #//decoded['lat_mins'] = minutes;
      
        lat_south = bytes[offset+3] & 0x01
        decoded['latitude'] = (-1 if lat_south  else 1) * degrees + minutes / 60
        offset += 4;
      
        degrees = (bytes[offset] >> 4) * 100 + (bytes[offset] & 0x0F) * 10 + (bytes[offset+1] >> 4) ;
          #//decoded['lng_deg'] = degrees;
      
        minutes = (bytes[offset+1] & 0x0F) * 10 + (bytes[offset+2] >> 4) + (bytes[offset+2] & 0x0F) / 10 + (bytes[offset+3] >> 4) / 100;
          #//decoded['lng_mins'] = minutes;
      
        lng_west = bytes[offset+3] & 0x01;
        decoded['longitude'] = (-1 if lng_west else 1) * degrees + minutes / 60;
        decoded['gps_reception'] = bytes[offset+4] >> 4;
        decoded['gps_satellites'] = bytes[offset+4] & 0x0F;
        offset += 5;
    
    
    if (bytes[0] & 0x08):  #// Uplink count present
        decoded['uplink_counter'] = bytes[offset+1];
    
    
    if (bytes[0] & 0x04): #// Downlink count present
        decoded['downlink_counter'] = bytes[offset+1];
    
    
    # battery voltage in mV
    if (bytes[0] & 0x02):  #// battery mV present
        decoded['battery'] = (bytes[offset] * 256 + bytes[offset+1]) / 1000;
        offset += 2;
    
    
    if (bytes[0] & 0x01):  #// RSSI and SNR present
        decoded['rssi'] = -bytes[offset+1]; #// rssi always negative
        decoded['snr'] =   bytes[offset] if (bytes[offset] < 128) else -(256 - bytes[offset]); #// snr in 2's complement
    

    return decoded;
# end Adeunis